# Project 1 Final

In [125]:
import plotly.express as px
import panel as pn
import pandas as pd
import quandl
import os
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import datetime as dt
import numpy as np

import hvplot.pandas
import plotly.express as px

%matplotlib inline

In [126]:
inflation_projection = quandl.get("FRED/PCECTPIMD")
inflation_projection.head()

,Value
Date,
2021-01-01,3.4
2022-01-01,2.1
2023-01-01,2.2


In [127]:
inflation_projection_plot = inflation_projection.hvplot.line()
inflation_projection_plot

:Curve   [Date]   (Value)

In [128]:
inflation_hist_df = quandl.get("RATEINF/CPI_USA")
inflation_hist_df

,Value
Date,
1913-01-31,9.800
1913-02-28,9.800
1913-03-31,9.800
1913-04-30,9.800
1913-05-31,9.700
...,...
2021-02-28,263.014
2021-03-31,264.877
2021-04-30,267.054


In [129]:
#dxy_var_hist_df = dxy_hist_df['Close'].var()
#dxy_var_hist_df
inflation_hist__df = inflation_hist_df[['Value']]
inflation_hist_pct_df = inflation_hist__df.pct_change()
inflation_hist_pct_df.dropna(inplace=True)
inflation_hist_pct_df

,Value
Date,
1913-02-28,0.000000
1913-03-31,0.000000
1913-04-30,0.000000
1913-05-31,-0.010204
1913-06-30,0.010309
...,...
2021-02-28,0.005474
2021-03-31,0.007083
2021-04-30,0.008219


In [130]:
# Rename the column
inflation_hist_pct_df.rename(columns = {"Value": "Inflation Rate"}, inplace = True)
inflation_hist_pct_df

,Inflation Rate
Date,
1913-02-28,0.000000
1913-03-31,0.000000
1913-04-30,0.000000
1913-05-31,-0.010204
1913-06-30,0.010309
...,...
2021-02-28,0.005474
2021-03-31,0.007083
2021-04-30,0.008219


In [131]:
cumulative_returns = (1 + inflation_hist_pct_df).cumprod() - 1
cumulative_returns_plot = cumulative_returns.hvplot.line()
cumulative_returns_plot

:Curve   [Date]   (Inflation Rate)

In [91]:
# import DXY history data to Pandas
dxy_hist_path = Path("./Resources/dxy-price-history.csv")
dxy_hist_df = pd.read_csv(dxy_hist_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
dxy_hist_df.index = pd.to_datetime(dxy_hist_df.index)
dxy_hist_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1971-08-24,117.839996,117.839996,117.839996,117.839996,117.839996,0.0
1971-08-25,117.839996,117.839996,117.839996,117.839996,117.839996,0.0
1971-08-26,117.839996,117.839996,117.839996,117.839996,117.839996,0.0
1971-08-27,117.839996,117.839996,117.839996,117.839996,117.839996,0.0
1971-08-29,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
# import tether history data to Pandas
tether_hist_path = Path("tether-price-history.csv")
tether_hist_df = pd.read_csv(tether_hist_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
tether_hist_df.index = pd.to_datetime(tether_hist_df.index)
tether_hist_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'tether-price-history.csv'

In [93]:
#import bitcoin history data to Pandas
btc_hist_path = Path("./Resources/bitcoin-price-history.csv")

btc_hist_df = pd.read_csv(btc_hist_path, index_col="Date",infer_datetime_format=True, parse_dates=True)
#btc_hist_df["Date"]=btc_hist_df["Date"].astype("datetime")
#btc_hist_df.dtypes
btc_hist_df.index = pd.to_datetime(btc_hist_df.index)
#btc_hist_df.set_index(pd.to_datetime(btc_hist_df.index))

btc_hist_df.sort_index(inplace=True)
btc_hist_df.head()

,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2013-07-17,96.7100,99.9700,96.1800,98.5000,0.0,1.120160e+09
2013-07-18,98.5000,98.8000,86.2000,90.5800,0.0,1.050191e+09
2013-07-19,90.0689,95.2001,87.8000,92.1700,0.0,1.035626e+09
2013-07-20,92.0000,93.1000,89.2711,89.3901,0.0,1.051175e+09
2013-07-21,89.8214,91.9500,88.0000,90.7573,0.0,1.025637e+09


In [94]:
# import eth history data to Pandas
eth_hist_path = Path("./Resources/ethereum-price-history.csv")
eth_hist_df = pd.read_csv(eth_hist_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
eth_hist_df.index = pd.to_datetime(eth_hist_df.index)

eth_hist_df.sort_index(inplace=True)
eth_hist_df.head()

,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2016-07-25,12.7495,13.8570,12.6838,13.8105,6.168861e+07,1.095915e+09
2016-07-26,13.8361,13.8511,11.7272,11.7632,5.788441e+07,1.056037e+09
2016-07-27,11.9895,13.3119,11.8357,12.9646,8.589149e+07,1.046191e+09
2016-07-28,12.9728,12.9882,12.4927,12.8619,3.914490e+07,1.050281e+09
2016-07-29,12.8436,12.8706,12.5542,12.8162,1.877377e+07,1.046257e+09


In [95]:
# import gld history data to Pandas
gld_hist_path = Path("./Resources/LBMA-GOLD.csv")
gld_hist_df = pd.read_csv(gld_hist_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
gld_hist_df.index = pd.to_datetime(gld_hist_df.index)

gld_hist_df.sort_index(inplace=True)
gld_hist_df.head()

,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM)
Date,,,,,,
1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN
1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN
1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN
1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN
1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN


In [96]:
#dxy_var_hist_df = dxy_hist_df['Close'].var()
#dxy_var_hist_df
dxy_hist_close_df = dxy_hist_df[['Close']]
dxy_hist_pct_df = dxy_hist_close_df.pct_change()
dxy_hist_pct_df.dropna(inplace=True)

dxy_hist_pct_df

,Close
Date,
1971-08-25,0.000000
1971-08-26,0.000000
1971-08-27,0.000000
1971-08-29,0.000000
1971-08-30,0.000000
...,...
2021-07-19,0.002158
2021-07-20,0.000861
2021-07-21,-0.002366


In [97]:
# Rename the column
dxy_hist_pct_df.rename(columns = {"Close": "DXY"}, inplace = True)
dxy_hist_pct_df

,DXY
Date,
1971-08-25,0.000000
1971-08-26,0.000000
1971-08-27,0.000000
1971-08-29,0.000000
1971-08-30,0.000000
...,...
2021-07-19,0.002158
2021-07-20,0.000861
2021-07-21,-0.002366


In [98]:
cumulative_returns = (1 + dxy_hist_pct_df).cumprod()

In [99]:
cumulative_returns_plot = cumulative_returns.hvplot(title= "DXY cumulative returns")

In [100]:
cumulative_returns_plot_dxy = cumulative_returns.tail(100).hvplot.violin(by= "index.week", 
                                                                         title = "DXY Cumulative returns. Last 100 days by weekly distribution")
cumulative_returns_plot + cumulative_returns_plot_dxy

/Users/dustinfeinberg/opt/anaconda3/lib/python3.8/site-packages/hvplot/util.py:413: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  extra_cols[var] = getattr(data.index, dt_str)


:Layout
   .Curve.I  :Curve   [Date]   (DXY)
   .Violin.I :Violin   [index.week]   (DXY)

In [101]:
#tether_var_hist_df = tether_hist_df['Close'].var()
#tether_var_hist_df
tether_hist_close_df = tether_hist_df[['Close']]
tether_hist_pct_df = tether_hist_close_df.pct_change()
tether_hist_pct_df.dropna(inplace=True)
tether_hist_pct_df

NameError: name 'tether_hist_df' is not defined

In [102]:
# Rename the column
tether_hist_pct_df.rename(columns = {"Close": "Tether"}, inplace = True)
tether_hist_pct_df

NameError: name 'tether_hist_pct_df' is not defined

In [103]:
cumulative_returns = (1 + tether_hist_pct_df).cumprod()
cumulative_returns_plot = cumulative_returns.hvplot.line(title = "Tether cumulative returns")
cumulative_returns_plot_tether = cumulative_returns.tail(100).hvplot.violin(by= "index.week",
                                                                  title = 'Tethers  Cumultive returns. Last 100 days by weekly distribution', figsize= "30,10")
cumulative_returns_plot + cumulative_returns_plot_tether

NameError: name 'tether_hist_pct_df' is not defined

In [104]:
#btc_var_hist_df = btc_hist_df['Close'].var()
#btc_var_hist_df
btc_hist_close_df = btc_hist_df[['Close']]
btc_hist_pct_df = btc_hist_close_df.pct_change()
btc_hist_pct_df.dropna(inplace=True)

In [105]:
# Rename the column
btc_hist_pct_df.rename(columns = {"Close": "BTC"}, inplace = True)
btc_hist_pct_df

,BTC
Date,
2013-07-18,-0.080406
2013-07-19,0.017554
2013-07-20,-0.030161
2013-07-21,0.015295
2013-07-22,0.009395
...,...
2021-07-19,-0.030209
2021-07-20,-0.035395
2021-07-21,0.080074


In [106]:
cumulative_returns = (1 + btc_hist_pct_df).cumprod()
cumulative_returns_plot = cumulative_returns.hvplot.line(title = "BTC Cumulative Returns")
#cumulative_returns_plot = cumulative_returns.hvplot.line()
cumulative_returns_plot_btc = cumulative_returns.tail(100).hvplot.violin(by= "index.week",
                                                                  title = 'BTC Cumultive returns. Last 100 days by weekly distribution', figsize= "30,10")
cumulative_returns_plot + cumulative_returns_plot_btc

/Users/dustinfeinberg/opt/anaconda3/lib/python3.8/site-packages/hvplot/util.py:413: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  extra_cols[var] = getattr(data.index, dt_str)


:Layout
   .Curve.I  :Curve   [Date]   (BTC)
   .Violin.I :Violin   [index.week]   (BTC)

In [107]:
cumulative_returns = (1 + btc_hist_pct_df).cumprod()


In [108]:
#eth_var_hist_df = eth_hist_df['Close'].var()
#eth_var_hist_df
eth_hist_close_df = eth_hist_df[['Close']]
eth_hist_pct_df = eth_hist_close_df.pct_change()
eth_hist_pct_df.dropna(inplace=True)

In [109]:
# Rename the column
eth_hist_pct_df.rename(columns = {"Close": "ETH"}, inplace = True)
eth_hist_pct_df

,ETH
Date,
2016-07-26,-0.148242
2016-07-27,0.102132
2016-07-28,-0.007922
2016-07-29,-0.003553
2016-07-30,-0.026506
...,...
2021-07-19,-0.040577
2021-07-20,-0.020670
2021-07-21,0.116298


In [110]:
cumulative_returns = (1 + eth_hist_pct_df).cumprod()
cumulative_returns_plot = cumulative_returns.hvplot.line(title= "ETH Cumulative Returns")
cumulative_returns_plot100 = cumulative_returns.tail(100).hvplot.violin(by= "index.week",
                                                                  title = 'ETH returns. Last 100 days by weekly analysis',
                                                                        figsize= "30,10", legend = True)
cumulative_returns_plot + cumulative_returns_plot100

/Users/dustinfeinberg/opt/anaconda3/lib/python3.8/site-packages/hvplot/util.py:413: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  extra_cols[var] = getattr(data.index, dt_str)


:Layout
   .Curve.I  :Curve   [Date]   (ETH)
   .Violin.I :Violin   [index.week]   (ETH)

In [111]:
#gld_var_hist_df = gld_hist_df['Close'].var()
#gld_var_hist_df
gld_hist_df = gld_hist_df[['USD (PM)']]
gld_hist_pct_df = gld_hist_df.pct_change()
gld_hist_pct_df.dropna(inplace=True)
gld_hist_pct_df

,USD (PM)
Date,
1968-04-02,-0.010610
1968-04-03,0.008043
1968-04-04,-0.017287
1968-04-05,0.001353
1968-04-08,0.001351
...,...
2021-06-24,-0.003768
2021-06-25,0.001008
2021-06-28,-0.003554


In [112]:
# Rename the column
gld_hist_pct_df.rename(columns = {"USD (PM)": "Gold"}, inplace = True)
gld_hist_pct_df

,Gold
Date,
1968-04-02,-0.010610
1968-04-03,0.008043
1968-04-04,-0.017287
1968-04-05,0.001353
1968-04-08,0.001351
...,...
2021-06-24,-0.003768
2021-06-25,0.001008
2021-06-28,-0.003554


In [113]:
cumulative_returns = (1 + gld_hist_pct_df).cumprod()
cumulative_returns_plot = cumulative_returns.hvplot.line(title= "Gold Cumulative Return")
cumulative_returns_plot_gold100 = cumulative_returns.tail(100).hvplot.violin(by= "index.week",
                                                                  title = 'Gold Cumulative Return. Last 100 days by weekly distribution',
                                                                        figsize= "30,10", legend = True)

cumulative_returns_plot + cumulative_returns_plot_gold100

/Users/dustinfeinberg/opt/anaconda3/lib/python3.8/site-packages/hvplot/util.py:413: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  extra_cols[var] = getattr(data.index, dt_str)


:Layout
   .Curve.I  :Curve   [Date]   (Gold)
   .Violin.I :Violin   [index.week]   (Gold)

In [114]:
# Concatenate all dataframes into one dataframe
df_daily = pd.concat([btc_hist_pct_df, eth_hist_pct_df, dxy_hist_pct_df, gld_hist_pct_df, tether_hist_pct_df], axis = "columns", join = "inner")

df_daily.sort_index(inplace=True)
df_daily.head()


NameError: name 'tether_hist_pct_df' is not defined

In [115]:
# Plot daily returns
df_daily.hvplot(figsize = (20,10), title= 'Daily Returns of BTC, ETH, DXY, Gold, Tether')

NameError: name 'df_daily' is not defined

In [116]:
# Box plot to visually illustrate risk
df_daily.hvplot(kind = "box", title= 'Risk Illustrator',figsize = (20, 10))

NameError: name 'df_daily' is not defined

In [117]:
# Plot cumulative returns
df_cum = (1 + df_daily).cumprod()
df_cum.hvplot(title= 'Cumulative Returnc of BTC, ETH, DXY, Gold, Tether',figsize = (20, 10))

    

NameError: name 'df_daily' is not defined

In [118]:
# Calculate std for each asset
df_daily_std = pd.DataFrame(df_daily.std()).rename(columns = {0:"std"})
df_daily_std

NameError: name 'df_daily' is not defined

In [119]:
#df_daily_std.hvplot()
df_daily_std.hvplot(kind= 'bar',title = ' Standard deviation of BTC, ETH, DXY, Gold, Tether')

NameError: name 'df_daily_std' is not defined

In [120]:
# Determine which portfolio is more risky than dxy
higher_std = df_daily_std[df_daily_std["std"] > df_daily_std.loc["DXY", "std"]]
higher_std

NameError: name 'df_daily_std' is not defined

In [121]:
# Calculate annualized std
df_annualized_std = df_daily_std*np.sqrt(252)
df_annualized_std

NameError: name 'df_daily_std' is not defined

In [122]:
# Calculate and plot rolling std for dxy for 21 day period
dxy_rolling_21 = df_daily[["DXY"]].rolling(window=21).std()
dxy_rolling_21.hvplot(figsize = (20, 10) , title = 'Rolling 21 day Standard Deviation for DXY')

NameError: name 'df_daily' is not defined

In [123]:
correlation_dxy = pd.DataFrame(df_daily.corr()).loc[:, "DXY"]
correlation_dxy [correlation_dxy==correlation_dxy[correlation_dxy<1].max()]

NameError: name 'df_daily' is not defined

In [124]:
df_daily.hvplot(kind="scatter", y = "BTC", x = "DXY", figsize = (20, 10), title= "BTC DXY ")

NameError: name 'df_daily' is not defined

In [43]:
df_daily.hvplot(title= 'ETH DXY', kind="scatter", y = "ETH", x = "DXY", figsize = (20, 10))

NameError: name 'df_daily' is not defined

In [44]:
df_daily.hvplot(title ='Gold DXY',kind="scatter", y = "Gold", x = "DXY", figsize = (20, 10))

NameError: name 'df_daily' is not defined

In [65]:
# Calculate Beta for bitcoin compared to dxy
rolling_covariance = df_daily['BTC'].rolling(window=60).cov(df_daily['DXY'])
rolling_variance = df_daily['DXY'].rolling(window=60).var()
rolling_beta = rolling_covariance / rolling_variance
rolling_beta.hvplot(figsize=(20, 10), title = 'Rolling 60 day beta of Bitcoin comapred to DXY')

:Curve   [Date]   (0)

In [66]:
# Calculate Beta for tether compared to dxy
rolling_covariance = df_daily['Tether'].rolling(window=60).cov(df_daily['DXY'])
rolling_variance = df_daily['DXY'].rolling(window=60).var()
rolling_beta = rolling_covariance / rolling_variance
rolling_beta.hvplot(figsize=(20, 10), title = 'Rolling 60 day beta of Tether comapred to DXY')

:Curve   [Date]   (0)

In [67]:
# Calculate Beta for eth compared to dxy
rolling_covariance = df_daily['ETH'].rolling(window=60).cov(df_daily['DXY'])
rolling_variance = df_daily['DXY'].rolling(window=60).var()
rolling_beta = rolling_covariance / rolling_variance
rolling_beta.hvplot(figsize=(20, 10), title = 'Rolling 60 day beta of ETH compared to DXY')

:Curve   [Date]   (0)

In [68]:
# Calculate Beta for gold compared to dxy
rolling_covariance = df_daily['Gold'].rolling(window=60).cov(df_daily['DXY'])
rolling_variance = df_daily['DXY'].rolling(window=60).var()
rolling_beta = rolling_covariance / rolling_variance
rolling_beta.hvplot(figsize=(20, 10), title = 'Rolling 60 day beta of Gold comapred to DXY')

:Curve   [Date]   (0)

In [71]:
df_daily_ewm = df_daily.ewm(span = 21, adjust=False).mean()
(1 + df_daily_ewm).cumprod().hvplot(title = 'Cumulative product of BTC, ETH, DXY, Gold, Tether',figsize=(20, 10))

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [72]:
# Annualized sharpe ratios
sharpe_ratios = df_daily.mean()*252/(df_daily.std()*np.sqrt(252))
sharpe_ratios

BTC       0.975890
ETH       0.475993
DXY       0.147619
Gold      0.560437
Tether   -0.484811
dtype: float64

In [75]:
sharpe_ratios.hvplot(kind = "bar", title = "Sharpe Ratios of BTC, ETH, DXY, Gold, Tether")

:Bars   [index]   (0)